In [27]:
import requests, json
import numpy as np
import pickle as pkl
import pandas as pd

import sys
sys.path.insert(1, '../../../scripts/')
from s3_support import *

URL = "https://secure.qgiv.com/admin/qgivadmin/statistics/export_feature_adoption.php"
KEY = "DSQR59VwyFhw21PKDF4K"

# Export data

In [28]:
payload = {'key': KEY}
rsp = requests.post(URL, data=payload)

In [29]:
pkl.dump(rsp.text, open('response.pkl', 'wb'))

In [30]:
rsp_obj = json.loads(rsp.text)

# Exploring

In [31]:
rsp_obj = json.loads(pkl.load(open("response.pkl", 'rb')))

In [32]:
data = rsp_obj[0]
for e in data.keys():
    print("{}".format(e))
    print("\t{} entries".format(len(data[e])))
    print("\t{}".format(data[e].keys()))
    print()

implementation_data
	27 entries
	dict_keys(['forms_with_notifications', 'orgs_with_notifications', 'service_integrations', 'custom_reports', 'merchant_accounts', 'embeds', 'promises', 'restrictions', 'custom_fields', 'pledges', 'recurring', 'events', 'events_donations', 'events_fields', 'events_promo', 'sms', 'donor_logins', 'thermometers', 'classifications', 'categories', 'registration_store', 'product_fields', 'shipping_address', 'nonfundraising_participants', 'teams', 'aggregator_page', 'category_fields'])

bidirectional_adoption_data
	18 entries
	dict_keys(['giftassist_transactions', 'restrictions', 'fielddata', 'pledge', 'recurring', 'events', 'events_donatiosn', 'events_fielddata', 'event_promo', 'dl_onetime', 'dl_recurring', 'classifications', 'categories', 'purchases', 'purchases_shipping', 'registrations_nonfundraising', 'registrations_subregistrants', 'teams'])



# Processing

## Implementation

In [33]:
for k in data['implementation_data'].keys():
    print("{}".format(k.replace('_', ' ')))
    try:
        len_k = len(data['implementation_data'][k])
    except:
        print("\tNone found...")
        continue
        
    try:
        count_key = [e for e in data['implementation_data'][k][0].keys() if 'count' in e][0]
        avg_k = "{:.2f}".format(np.mean([int(e[count_key]) for e in data['implementation_data'][k]]))
    except:
        avg_k = None
    
    print("\tlen: {}; average: {}".format(len_k, avg_k))
    print("\trow: {}".format(data['implementation_data'][k][0]))

forms with notifications
	len: 1; average: None
	row: 0
orgs with notifications
	len: 80; average: None
	row: 0
service integrations
	len: 796; average: 1.41
	row: {'org': '6', 'count(id)': '1'}
custom reports
	len: 681; average: 4.19
	row: {'org': '0', 'count(id)': '3'}
merchant accounts
	len: 94949; average: 1.00
	row: {'form': '1', 'paymentMethods': 'AM', 'count(id)': '1'}
embeds
	len: 2350; average: 3.80
	row: {'org': '6', 'count(id)': '1'}
promises
	len: 4; average: 8.50
	row: {'form': '947632', 'count(id)': '3'}
restrictions
	len: 6229; average: 7.65
	row: {'form': '0', 'count(id)': '2'}
custom fields
	len: 14442; average: 3.45
	row: {'form': '0', 'count(id)': '12'}
pledges
	len: 1549; average: 3.80
	row: {'form': '0', 'count(id)': '9'}
recurring
	len: 7858; average: 3.59
	row: {'org': '6', 'count(id)': '8'}
events
	len: 9912; average: 5.59
	row: {'form': '0', 'count(id)': '7'}
events donations
	len: 6705; average: 5.19
	row: {'form': '0', 'count(id)': '7'}
events fields
	len: 63

## Bidirectional Adoption Data

In [34]:
for k in data['bidirectional_adoption_data'].keys():
    print("{}".format(k.replace("_", " ")))
    
    try:
        len_k = len(data['bidirectional_adoption_data'][k])
    except:
        print("\tNone found...")
        continue
        
    try:
        count_key = [e for e in data['bidirectional_adoption_data'][k][0].keys() if 'count' in e][0]
        avg_k = "{:.2f}".format(np.mean([int(e[count_key]) for e in data['bidirectional_adoption_data'][k]]))
    except:
        avg_k = None
    
    print("\tlen: {}; average: {}".format(len_k, avg_k))
    print("\trow: {}".format(data['bidirectional_adoption_data'][k][-1]))

giftassist transactions
	len: 7972; average: 96.40
	row: {'form': '956639', 'count(distinct ga.id)': '1'}
restrictions
	len: 4072; average: 735.47
	row: {'form': '956616', 'count(distinct t.id)': '23'}
fielddata
	len: 2135; average: 467.50
	row: {'form': '956395', 'count(distinct fld.id)': '1'}
pledge
	len: 251; average: 2.47
	row: {'form': '956207', 'count(distinct td.pledge)': '1'}
recurring
	len: 6079; average: 415.92
	row: {'form': '956612', 'count(id)': '4'}
events
	len: 5735; average: 170.54
	row: {'form': '956481', 'count(distinct te.transaction)': '3'}
events donatiosn
	len: 20497; average: 373.16
	row: {'form': '956639', 'count(distinct td.id)': '2'}
events fielddata
	len: 2987; average: 952.90
	row: {'form': '956271', 'count(distinct pd.id)': '55'}
event promo
	len: 520; average: 96.36
	row: {'form': '661571', 'count(distinct p.id)': '1'}
dl onetime
	len: 8565; average: 86.98
	row: {'form': '956627', 'count(ta.id)': '1'}
dl recurring
	len: 2051; average: 176.86
	row: {'form':

## Formatting

In [35]:
len_orgs = redshift_query_read("select count(id) as count from organization where status='1'", schema='production')['count'].iloc[0]
len_forms = redshift_query_read("select count(id) as count from form where status='1'", schema="production")['count'].iloc[0]

In [36]:
imp_data = data['implementation_data']
bid_data = data['bidirectional_adoption_data']

In [37]:
def get_org_data(data_set):
    count_key = [e for e in data_set[0].keys() if 'count' in e][0]
    avg_value = "{:.2f}".format(np.mean([int(e[count_key]) for e in data_set]))
    
    percentage_text = "{:.2f}%".format((float(len(data_set)) / float(len_orgs)) * 100)
    note_text = "{} orgs; {} per org".format(len(data_set), avg_value)
    
    return {'value': percentage_text, 'note': note_text}


def get_form_data(data_set):
    count_key = [e for e in data_set[0].keys() if 'count' in e][0]
    avg_value = "{:.2f}".format(np.mean([int(e[count_key]) for e in data_set]))
    
    percentage_text = "{:.2f}%".format((float(len(data_set)) / float(len_forms)) * 100)
    note_text = "{} forms; {} per form".format(len(data_set), avg_value)
    
    return {'value': percentage_text, 'note': note_text}

In [38]:
adoption_data = {}
all_keys = list(set(list(imp_data.keys()) + list(bid_data.keys())))

for k in all_keys:
    imp_entry = {}
    bid_entry = {}
    inst_entry = {}
    
    if k in imp_data and imp_data[k] is not None:
        if type(imp_data[k]) == str:
            imp_entry = {'count': imp_data[k]}
        elif type(imp_data[k][0]) == str:
            imp_entry = {'count': len(imp_data[k])}
        elif 'org' in imp_data[k][0].keys():
            imp_entry = get_org_data(imp_data[k])
        elif 'form' in imp_data[k][0].keys():
            imp_entry = get_form_data(imp_data[k])
    if k in bid_data and bid_data[k] is not None:
        if 'org' in bid_data[k][0].keys():
            bid_entry = get_org_data(bid_data[k])
        elif 'form' in bid_data[k][0].keys():
            bid_entry = get_form_data(bid_data[k])
            
        
    adoption_data[k] = {
        'implemented': imp_entry,
        'bidirectional': bid_entry,
        'institutional': inst_entry
    }

In [39]:
pkl.dump(adoption_data, open("adoption_data.pkl", "wb"))

In [40]:
adoption_data

{'pledges': {'implemented': {'value': '7.84%',
   'note': '1549 forms; 3.80 per form'},
  'bidirectional': {},
  'institutional': {}},
 'dl_onetime': {'implemented': {},
  'bidirectional': {'value': '43.33%', 'note': '8565 forms; 86.98 per form'},
  'institutional': {}},
 'categories': {'implemented': {'value': '28.90%',
   'note': '5713 forms; 3.29 per form'},
  'bidirectional': {'value': '20.48%', 'note': '4048 forms; 143.77 per form'},
  'institutional': {}},
 'restrictions': {'implemented': {'value': '31.51%',
   'note': '6229 forms; 7.65 per form'},
  'bidirectional': {'value': '20.60%', 'note': '4072 forms; 735.47 per form'},
  'institutional': {}},
 'events_fields': {'implemented': {'value': '31.88%',
   'note': '6303 forms; 27.61 per form'},
  'bidirectional': {},
  'institutional': {}},
 'classifications': {'implemented': {'value': '13.27%',
   'note': '2623 forms; 8.54 per form'},
  'bidirectional': {'value': '7.93%', 'note': '1568 forms; 168.95 per form'},
  'institutional':

# Integrating Segment data

CMS, widgets, and goals

In [41]:
q = '''select
            count(distinct(users.org)) as orgs
        from saved_page
            left join users on saved_page.uuid=users.uuid'''
cms_orgs = redshift_query_read(q, schema='secure')['orgs'].iloc[0]

In [42]:
q = '''select
            count(distinct(users.org)) as orgs
        from created_widget
            left join users on created_widget.uuid=users.uuid'''
widget_orgs = redshift_query_read(q, schema='secure')['orgs'].iloc[0]

In [43]:
q = '''select
            count(distinct(users.org)) as orgs
        from goals_saved_item_settings
            left join users on goals_saved_item_settings.uuid=users.uuid'''
goals_orgs = redshift_query_read(q, schema="secure")['orgs'].iloc[0]

In [44]:
print("Engaged CMS orgs: {}".format(cms_orgs))
print("Engaged widget orgs: {}".format(widget_orgs))
print("Engaged goals orgs: {}".format(goals_orgs))

Engaged CMS orgs: 212
Engaged widget orgs: 69
Engaged goals orgs: 24


In [45]:
len_orgs = redshift_query_read("select count(id) as count from organization where status='1'", schema='production')['count'].iloc[0]

perc_goals = float(goals_orgs) / float(len_orgs)
perc_widgets = float(widget_orgs) / float(len_orgs)
perc_cms = float(cms_orgs) / float(len_orgs)

In [48]:
adoption_data['goals'] = {
    'implemented': {
        'value': perc_goals,
        'note': '{} orgs'.format(goals_orgs)
    },
    'bidirectional': {},
    'institutional': {}
}
adoption_data['widgets'] = {
    'implemented': {
        'value': perc_widgets,
        'note': '{} orgs'.format(widget_orgs)
    },
    'bidirectional': {},
    'institutional': {}
}
adoption_data['cms'] = {
    'implemented': {
        'value': perc_cms,
        'note': '{} orgs'.format(cms_orgs)
    },
    'bidirectional': {},
    'institutional': {}
}

In [49]:
adoption_data

{'pledges': {'implemented': {'value': '7.84%',
   'note': '1549 forms; 3.80 per form'},
  'bidirectional': {},
  'institutional': {}},
 'dl_onetime': {'implemented': {},
  'bidirectional': {'value': '43.33%', 'note': '8565 forms; 86.98 per form'},
  'institutional': {}},
 'categories': {'implemented': {'value': '28.90%',
   'note': '5713 forms; 3.29 per form'},
  'bidirectional': {'value': '20.48%', 'note': '4048 forms; 143.77 per form'},
  'institutional': {}},
 'restrictions': {'implemented': {'value': '31.51%',
   'note': '6229 forms; 7.65 per form'},
  'bidirectional': {'value': '20.60%', 'note': '4072 forms; 735.47 per form'},
  'institutional': {}},
 'events_fields': {'implemented': {'value': '31.88%',
   'note': '6303 forms; 27.61 per form'},
  'bidirectional': {},
  'institutional': {}},
 'classifications': {'implemented': {'value': '13.27%',
   'note': '2623 forms; 8.54 per form'},
  'bidirectional': {'value': '7.93%', 'note': '1568 forms; 168.95 per form'},
  'institutional':